In [1]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras import layers, Model
from sklearn.metrics import accuracy_score

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

x_train = np.expand_dims(x_train, axis=-1)  # Shape: (60000, 28, 28, 1)
x_test = np.expand_dims(x_test, axis=-1)    # Shape: (10000, 28, 28, 1)

# Create dictionary mapping labels to images
train_classes = {i: x_train[y_train == i] for i in range(10)}
test_classes = {i: x_test[y_test == i] for i in range(10)}

# Function to create pairs of images (One-Shot Learning)
def create_pairs(classes, num_pairs=1000):
    pairs, labels = [], []
    for _ in range(num_pairs):
        label = random.choice(list(classes.keys()))
        img1, img2 = random.sample(list(classes[label]), 2)  # Same class
        pairs.append([img1, img2])
        labels.append(1)  # Similar pair

        diff_label = random.choice([l for l in classes.keys() if l != label])
        img3 = random.choice(classes[diff_label])  # Different class
        pairs.append([img1, img3])
        labels.append(0)  # Dissimilar pair

    return np.array(pairs), np.array(labels)

# Create training and testing pairs
train_pairs, train_labels = create_pairs(train_classes, 5000)
test_pairs, test_labels = create_pairs(test_classes, 1000)

# CNN Feature Extractor for Siamese Network
def build_siamese_encoder():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu')
    ])
    return model

# Siamese Model using distance between embeddings
def build_siamese_network():
    input_a = layers.Input(shape=(28,28,1))
    input_b = layers.Input(shape=(28,28,1))

    encoder = build_siamese_encoder()
    encoded_a = encoder(input_a)
    encoded_b = encoder(input_b)

    distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([encoded_a, encoded_b])
    output = layers.Dense(1, activation='sigmoid')(distance)

    return Model(inputs=[input_a, input_b], outputs=output)

# Compile & Train the Siamese Network
siamese_net = build_siamese_network()
siamese_net.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
siamese_net.fit([train_pairs[:,0], train_pairs[:,1]], train_labels, epochs=10, batch_size=32)

# Evaluate Model
preds = siamese_net.predict([test_pairs[:,0], test_pairs[:,1]])
pred_labels = (preds > 0.5).astype(int)
accuracy = accuracy_score(test_labels, pred_labels)

print(f"Siamese Network Accuracy: {accuracy * 100:.2f}%")


c:\Users\anand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.7057 - loss: 0.5217
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9176 - loss: 0.2087
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9643 - loss: 0.1066
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9831 - loss: 0.0546
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9859 - loss: 0.0430
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9925 - loss: 0.0255
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9951 - loss: 0.0153
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9967 - loss: 0.0107
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9920 - loss: 0.0261
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9972 - loss: 0.0088
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Siamese Network Accuracy: 94.90%
